In [ ]:
# Import the required libraries

# Import OS for file handling
import os
# Import random for randomizing the images 
import random
# Import pandas for dataset management
import pandas as pd
# Import matplotlib for reading and displaying images 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import numpy for numeric operation
import numpy as np
# Import skimage.io for reading and displaying images
from skimage.io import imread
# Import skimage.transform for resizing the image
from skimage.transform import resize

In [ ]:
# Create an array to store target(labels), images and flat data
target = []
# Create an array to store flat data
flat_data = []
# Create an array to store images
images = []

# Initialize the data directory
DataDirectory = 'C:/Users/nkhanal/Downloads/AnimalClassification/train'

# List the data directory as category
Categories = os.listdir(DataDirectory)


In [ ]:
# Loop through the category
for i in Categories:
    # Display each category and its respective label
    print("Category is:",i,"\tLabel encoded as:",Categories.index(i))
    # Create a class for each labels
    target_class = Categories.index(i)
    # Create data path for all folders
    path = os.path.join(DataDirectory,i)
    # Read and resize image to same dimensions
    for img in os.listdir(path):
        # Read the image using skimage
        img_array = imread(os.path.join(path,img),plugin='matplotlib')
        # Resize the image from the image array
        img_resized = resize(img_array,(150,150,3))
        # Flatten the resized image
        flat_data.append(img_resized.flatten())
        # Add the resized image to image array
        images.append(img_resized)
        # Add the label class to target array
        target.append(target_class)
# Convert arrays to numpy array format
flat_data = np.array(flat_data)
images = np.array(images)
target = np.array(target)

In [ ]:
# Randomize the sample to be displayed
selected_animals = random.sample(Categories, 2)

# Set up the matplotlib figure and axes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Iterate over the selected animals and display one image from each folder
for i, animal in enumerate(selected_animals):
    # Create a full path to the animal directory
    animal_directory = os.path.join(DataDirectory, animal)
    # List all the files available in directory
    animal_files = os.listdir(animal_directory)
    # If files are detected
    if animal_files:
        # Randomize the detected files
        random_animal_file = random.choice(animal_files)
        # Add it to the image path
        animal_image_path = os.path.join(animal_directory, random_animal_file)
        # Read the random images using matplotlib mpimg
        animal_image = mpimg.imread(animal_image_path)
        # Display the images
        axes[i].imshow(animal_image)
        # Display their labels
        axes[i].set_title(animal)
        # Disable the plot axes
        axes[i].axis("off")

# Adjust the spacing between subplots
plt.tight_layout()
# Display the figure
plt.show()

In [ ]:
# Create a dataset for the flat data
df = pd.DataFrame(flat_data)
# Create a column for output data called Target
df['Target'] = target

In [ ]:
# Split the dataset into training and validation

# Import the training and testing split module
from sklearn.model_selection import train_test_split

# Get the images from the dataset
x = df.iloc[:,:-1].values
# Get the labels from the dataset
y = target

# Split the images and labels into 80% training and 20% validation datasets
X_train, X_test, y_train, y_test = train_test_split(
    x, 
    y, 
    test_size=0.20, 
    shuffle=True,
    random_state=42,
)

In [ ]:
# Apply Support Vector Machine classifier

# Import the GridSearchCv for selecting the most effective hyperparameters
from sklearn.model_selection import GridSearchCV
# Import the support vector machine classifier
from sklearn.svm import SVC

# Set the parameters for testing
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
# Create an instance of SVC class
svc = SVC()
# Apply the parameters for finding the best parameters
clf = GridSearchCV(svc, param_grid)
# Train the model to find best parameters
clf.fit(X_train,y_train)

In [ ]:
# Display parameters selected by GridSearchCV 
print("Best parameters to apply are:",clf.best_params_)
# Display model after hyperparameter tuning
svm = clf.best_estimator_
print("Model after tuning is:\n",svm)

In [ ]:
# Predict the output of model by applying the best parameter
y_prediction = svm.predict(X_test)

In [ ]:
# Evaluate the model using confusion matrix, classification report and accuracy
from sklearn.metrics import accuracy_score
print("Accuracy score:",100*accuracy_score(y_prediction,y_test))

In [ ]:
# Import the pickle module for saving the generated machine learning model
import pickle
# Save SVM model in pickle file
pickle.dump(svm,open("C:/Users/nkhanal/Downloads/Classification_Model.p","wb"))

In [ ]:
# Read byte from the saved ML model
test_model = pickle.load(open("C:/Users/nkhanal/Downloads/Classification_Model.p","rb"))

In [ ]:
# Test images using the generated ML model

# Define the test path
test_path = 'C:/Users/nkhanal/Downloads/AnimalClassification/test'
# List the folders on the test path
test_animals = os.listdir(test_path)
for j in range (2):
    for i in test_animals:
        # Set a flat data array
        flat_data = []
        # Set different sub-path through each iteration
        path = os.path.join(test_path,i)
        # Randomize the selected paths to select different images
        random_image = random.choice(os.listdir(path))
        # Set the random image path
        image_path = os.path.join(path, random_image)
        # Read the image from the created path
        img_array = imread(image_path, plugin='matplotlib')
        # Resize the image
        img_resized = resize(img_array, (150, 150, 3))
        # Flatten the resized image for testing
        flat_data.append(img_resized.flatten())
        # Display the image
        plt.imshow(img_resized)
        # Predict the image using the generated model
        y_output = test_model.predict(flat_data)
        # Set the label of the predicted output
        y_output = Categories[y_output[0]]
        # Display the image
        plt.imshow(img_resized)
        plt.axis('off')
        plt.show()
        # Display the predicted output label
        print("PREDICTED OUTPUT IS:",y_output)